In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt

In [2]:
def read_ds(ds_name: str):
  fileName = 'Data/'+ds_name+'.csv' 
  with open(fileName, 'r', encoding='utf-8') as f:
    lignes = [line.strip().split(',') for line in f]

  # Trouver le nombre maximal de colonnes
  max_cols = max(len(l) for l in lignes)

  # Normaliser les lignes plus courtes
  for l in lignes:
      l += [''] * (max_cols - len(l))

  # Créer les noms de colonnes
  colonnes = ['util', 'navigateur'] + [f'col{i}' for i in range(3, max_cols + 1)]

  # Créer le DataFrame
  df = pd.DataFrame(lignes, columns=colonnes)

  return df

In [3]:
features_train = read_ds("train")
features_test = read_ds("test")
print(features_train.shape)
print(features_test.shape)
features_train.head()

(3279, 14470)
(324, 7726)


,util,navigateur,col3,col4,col5,col6,col7,col8,col9,col10,...,col14461,col14462,col14463,col14464,col14465,col14466,col14467,col14468,col14469,col14470
0,nuh,Firefox,Création d'un écran(infologic.core.accueil.Acc...,Affichage d'une dialogue,Exécution d'un bouton,Fermeture d'une dialogue,Affichage d'une dialogue,Exécution d'un bouton,Fermeture d'une dialogue,Création d'un écran(infologic.core.gui.control...,...,,,,,,,,,,
1,muz,Google Chrome,Création d'un écran(infologic.core.gui.control...,Création d'un écran(infologic.core.gui.control...,t5,Sélection d’un onglet(infologic.orga.modules.O...,t10,Exécution d'un bouton,t15,Sélection d’un onglet,...,,,,,,,,,,
2,zrx,Microsoft Edge,Affichage d'une dialogue(infologic.core.gui.co...,Exécution d'un bouton,Chainage,Fermeture d'une dialogue,Affichage d'une dialogue(infologic.acti.module...,Clic sur une grille d'historique de recherche,Raccourci,Fermeture d'une dialogue,...,,,,,,,,,,
3,pou,Firefox,Création d'un écran(infologic.core.gui.control...,t5,Exécution d'un bouton(MAINT),Affichage d'une dialogue,Fermeture d'une dialogue,Double-clic,Exécution d'un bouton,Lancement d'une stat(infologic.core.gui.contro...,...,,,,,,,,,,
4,ald,Google Chrome,Affichage d'une dialogue(infologic.acti.module...,t5,Exécution d'un bouton,Fermeture d'une dialogue,t10,Entrée en saisie dans un formulaire,t10,Affichage d'une dialogue,...,,,,,,,,,,


In [4]:
actions_par_util = features_train['util'].value_counts()
actions_par_util

util
skm    75
slq    71
cjr    46
flj    42
hjs    37
       ..
rff     4
bez     4
crn     4
azn     4
fyg     4
Name: count, Length: 247, dtype: int64

In [5]:
import warnings
from IPython.display import display, Markdown

# décorateurs utilitaires pour supprimer les avertissements de la sortie et imprimer un cadre de données dans un tableau Markdown.
def ignore_warnings(f):
    def _f(*args, **kwargs):
        warnings.filterwarnings('ignore')
        v = f(*args, **kwargs)
        warnings.filterwarnings('default')
        return v
    return _f

# affiche un DataFrame Pandas sous forme de tableau Markdown dans un notebook Jupyter.
def markdown_table(headNtail=False, use_index=True, title=None, precision=2):
    def _get_value(val): return str(round(val, precision) if isinstance(val, float) else val)
    def _format_row(row): 
        row_str = ""
        if use_index: row_str += f"|{str(row.name)}"
        for value in row.values: row_str += f"| {_get_value(value)}"
        return row_str + "|"
    def _get_str(df):
        return "\n".join(df.apply(_format_row, axis=1))
    def _deco(f):
        def _f(*args, **kwargs):
            df = f(*args, **kwargs)
            _str = f"#### {title}\n" if title else ""
            header = ([str(df.index.name)] if use_index else []) + df.columns.astype(str).to_list() 
            _str += f"|{'|'.join(header)}|" + f"\n|{'--|'*len(header)}\n" if header else None
            if headNtail:
                _str += _get_str(df.head())
                _str += "\n|...|...|\n"
                _str += _get_str(df.tail())
            else:
                _str += _get_str(df)
            display(Markdown(_str))
        return _f
    return _deco

# fonction utilitaire permettant d'obtenir une grille graphique à partir d'un nombre arbitraire de lignes/colonnes ou de données.
def get_grid(n, n_row=None, n_col=None, titles=None, figsize=(10, 8), wspace=.5, hspace=.5, **kwargs):
    if n_row: n_col= n_col or math.floor(n/n_row)
    elif n_col: n_row= n_row or math.ceil(n/n_col)
    else:
        n_row = math.ceil(math.sqrt(n))
        n_col = math.floor(n/n_row)
    fig, axs = plt.subplots(n_row, n_col, figsize=figsize, **kwargs)
    plt.subplots_adjust(hspace=hspace, wspace=wspace)
    if titles is not None:
        for ax, title in zip(axs.flat, titles): ax.set_title(title)
    return fig, axs

In [6]:
@markdown_table(title="Navigateur par utilisateur", headNtail=True)
def browsers_per_player(df):
    """
    Pour chaque utilisateur (en ligne), compter combien de fois il a utilisé chaque navigateur (en colonne)
    """
    # groupby + pivot pour obtenir un tableau utilisateurs × navigateurs
    table = df.groupby(['util', 'navigateur']).size().unstack(fill_value=0)
    return table  # <- très important pour que le décorateur affiche le tableau

browsers_per_player(features_train)


#### Navigateur par utilisateur
|util|Firefox|Google Chrome|Microsoft Edge|Opera|
|--|--|--|--|--|
|aho| 0| 10| 0| 0|
|ajo| 13| 0| 0| 0|
|akx| 0| 14| 0| 0|
|ald| 0| 15| 0| 0|
|ats| 12| 0| 0| 0|
|...|...|
|zqs| 0| 0| 16| 0|
|zro| 13| 0| 0| 0|
|zrx| 0| 0| 12| 0|
|zus| 13| 0| 0| 0|
|zyk| 0| 21| 0| 0|

In [7]:
@markdown_table(headNtail=True, title="Stats: Distribution des utilisateurs")
def get_Y_stats(df):
    """
    Inspecte la distribution de la variable dépendante Y.
    """
    # Vérifier que la colonne 'Y' existe
    if 'util' not in df.columns:
        raise ValueError("La colonne 'Y' n'existe pas dans le DataFrame")

    # Distribution de Y : nombre d'occurrences de chaque valeur
    y_counts = df['util'].value_counts().to_frame(name='Nb_occurrences')

    # Pourcentage de chaque classe
    y_counts['%'] = (y_counts['Nb_occurrences'] / len(df) * 100).round(2)

    return y_counts

get_Y_stats(features_train)



#### Stats: Distribution des utilisateurs
|util|Nb_occurrences|%|
|--|--|--|
|skm| 75.0| 2.29|
|slq| 71.0| 2.17|
|cjr| 46.0| 1.4|
|flj| 42.0| 1.28|
|hjs| 37.0| 1.13|
|...|...|
|rff| 4.0| 0.12|
|bez| 4.0| 0.12|
|crn| 4.0| 0.12|
|azn| 4.0| 0.12|
|fyg| 4.0| 0.12|